<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=47e22bd16d56431d48556f6babe2012e962a0e7aa9520dd1f1212be46f9e8014
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-04 09:48:01
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: -7.69 K (-0.05%)
Current PnL: -19.28 L (-12.63%)
CY Booked + Current PnL: -6.63 L (-4.35%)
-------------------
Total profit:  2.34 L
Total loss:  -21.62 L
-------------------
Total Booked + Current PnL: 20.35 L (16.2%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.71%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.26 L (59.38%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,68.0,X-LC,5.53,231707.0,21341.0,7438.0,0.04,10.14,3.21,13.68,37.0,2.87,1.61,28.34,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,105.76,56.0,M-SC,1.74,88622.0,-12155.0,12248.0,-0.32,-12.06,13.82,0.09,245.0,-0.99,0.61,16.43,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.17,171835.0,30995.0,13489.0,0.17,22.01,7.85,31.59,66.0,2.30,1.19,54.32,XY24,NTT,BANKS
33,ICICIGI,2252.93,-12.68,63.0,X-MC,5.23,150109.0,13976.0,16602.0,1.04,10.27,11.06,22.47,91.0,0.84,1.04,24.58,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.86,40.0,X-MC,2.51,88155.0,-2523.0,17428.0,1.21,-2.78,19.77,16.44,101.0,-0.14,0.61,16.83,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-51.56,77.0,H-SC,6.78,277698.0,-8807.0,266146.0,2.56,-3.07,95.84,89.82,134.0,-0.03,1.93,46.92,XY24,NTT,FINANCE
60,ROUTE,2236.84,-54.32,43.0,H-SC,22.48,74942.0,-58124.0,150978.0,2.51,-43.68,201.46,69.78,118.0,-0.38,0.52,4.18,SR,ATH,IT
32,HONAUT,58357.33,-20.29,61.0,X-SC,1.11,112410.0,-15528.0,62657.0,2.34,-12.14,55.74,36.84,143.0,-0.25,0.78,15.08,X40N,ATH,ELECTRICAL
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
61,SAIL,228.00,53.17,64.0,M-MC,9.84,241407.0,16445.0,151217.0,1.61,7.31,62.64,74.53,192.0,0.11,1.67,41.92,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,KANSAINER,340.00,-67.79,45.0,H-SC,2.63,221472.0,-48195.0,84536.0,-1.23,-17.87,38.17,13.47,138.0,-0.57,1.54,12.61,XY24,NTT,PAINTS
16,CAMS,4762.00,-5.16,48.0,X-SC,0.86,254291.0,2435.0,60013.0,-1.14,0.97,23.60,24.79,122.0,0.04,1.76,23.86,X40N,NTT,MISC
55,RAJOOENG,143.10,-42.78,44.0,H-SC,17.01,86680.0,-15820.0,56420.0,-1.12,-15.43,65.09,39.61,114.0,-0.28,0.60,5.86,AR,ATH,MISC
76,TITAGARH,1548.00,-4.29,52.0,H-SC,3.67,204630.0,-37547.0,148316.0,-1.04,-15.50,72.48,45.74,158.0,-0.25,1.42,33.09,XY24,NTT,ENGINEERING
29,HCLTECH,1922.01,4.95,59.0,X-LC,5.77,239849.0,-2075.0,61905.0,-1.02,-0.86,25.81,24.73,8.0,-0.03,1.66,14.25,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.76,56.0,M-SC,1.74,88622.0,-12155.0,12248.0,-0.32,-12.06,13.82,0.09,245.0,-0.99,0.61,16.43,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,45.0,H-SC,2.63,221472.0,-48195.0,84536.0,-1.23,-17.87,38.17,13.47,138.0,-0.57,1.54,12.61,XY24,NTT,PAINTS
17,CERA,9475.0,-21.24,56.0,H-SC,1.94,143382.0,-32521.0,74544.0,-0.78,-18.49,51.99,23.89,149.0,-0.44,0.99,23.83,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-23.71,47.0,H-MC,6.01,103227.0,-28608.0,70834.0,-0.02,-21.70,68.62,32.03,98.0,-0.40,0.72,15.62,OX40N,NTT,IT
67,SIS,528.0,2017.10,44.0,H-SC,2.99,85438.0,-25594.0,49204.0,0.59,-23.05,57.59,21.26,156.0,-0.52,0.59,15.43,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,67.0,M-LC,10.17,171835.0,30995.0,13489.0,0.17,22.01,7.85,31.59,66.0,2.3,1.19,54.32,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,53.17,64.0,M-MC,9.84,241407.0,16445.0,151217.0,1.61,7.31,62.64,74.53,192.0,0.11,1.67,41.92,XY24,BTT,STEEL
56,RECLTD,446.00,47.87,55.0,M-LC,4.93,208175.0,5005.0,37118.0,-0.18,2.46,17.83,20.74,55.0,0.13,1.44,9.49,XY25,NTT,FINANCE
31,HINDZINC,730.22,27.24,48.0,M-LC,10.72,207190.0,2114.0,110453.0,-0.91,1.03,53.31,54.89,52.0,0.02,1.44,25.38,X5K,ATH,METALS
46,KPIGREEN,731.05,13.63,74.0,H-SC,6.55,135261.0,9964.0,48964.0,-0.38,7.95,36.20,47.03,141.0,0.20,0.94,69.36,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.17,171835.0,30995.0,13489.0,0.17,22.01,7.85,31.59,66.0,2.30,1.19,54.32,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,13.63,74.0,H-SC,6.55,135261.0,9964.0,48964.0,-0.38,7.95,36.20,47.03,141.0,0.20,0.94,69.36,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.65,126610.0,11812.0,82233.0,-0.15,10.29,64.95,81.92,223.0,0.14,0.88,49.89,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.07,130634.0,7298.0,124324.0,-0.08,5.92,95.17,106.72,119.0,0.06,0.91,30.68,AR,ATH,MISC
31,HINDZINC,730.22,27.24,48.0,M-LC,10.72,207190.0,2114.0,110453.0,-0.91,1.03,53.31,54.89,52.0,0.02,1.44,25.38,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,42.0,M-LC,5.37,148831.0,-2114.0,111564.0,-0.19,-1.40,74.96,72.51,53.0,-0.02,1.03,4.54,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88852.0,-39818.0,85120.0,0.00,-30.95,95.80,35.20,219.0,-0.47,0.62,1.98,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-21.71,34.0,X-MC,4.84,196391.0,-3956.0,34467.0,0.34,-1.97,17.55,15.23,178.0,-0.11,1.36,29.64,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-12.96,34.0,X-LC,7.02,244600.0,-12463.0,47599.0,-0.57,-4.85,19.46,13.67,24.0,-0.26,1.70,14.57,XY25,NTT,FMCG
23,DMART,5391.45,-17.97,36.0,X-LC,8.65,208995.0,-1412.0,60588.0,0.58,-0.67,28.99,28.12,38.0,-0.02,1.45,22.80,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.50,51.0,X-LC,1.16,201566.0,1428.0,19471.0,-0.42,0.71,9.66,10.44,4.0,0.07,1.40,6.32,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.04,46.0,H-LC,1.23,155750.0,-30345.0,77828.0,-0.40,-16.31,49.97,25.51,15.0,-0.39,1.08,14.78,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.68,39.0,X-MC,1.28,200835.0,15.0,67782.0,-0.01,0.01,33.75,33.76,80.0,0.00,1.39,5.21,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.32,46.0,X-MC,2.90,312795.0,-15196.0,121740.0,-0.52,-4.63,38.92,32.48,92.0,-0.12,2.17,6.26,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.81,75.0,X-LC,4.25,249606.0,15462.0,33173.0,0.25,6.60,13.29,20.77,86.0,0.47,1.73,13.98,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,40.0,X-SC,37.70,51421.0,-13585.0,164511.0,-0.26,-20.90,319.93,232.18,198.0,-0.08,0.36,0.30,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88852.0,-39818.0,85120.0,0.00,-30.95,95.80,35.20,219.0,-0.47,0.62,1.98,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-28.15,38.0,X-MC,7.04,81140.0,-2370.0,22403.0,-0.47,-2.84,27.61,23.98,82.0,-0.11,0.56,2.82,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.08,44.0,X-MC,7.34,219460.0,-43905.0,153227.0,-0.26,-16.67,69.82,41.51,84.0,-0.29,1.52,2.99,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-14.69,45.0,X-MC,6.09,179777.0,-20729.0,117880.0,-0.26,-10.34,65.57,48.45,90.0,-0.18,1.25,3.67,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,40.0,X-SC,37.70,51421.0,-13585.0,164511.0,-0.26,-20.90,319.93,232.18,198.0,-0.08,0.36,0.30,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.15,38.0,X-MC,7.04,81140.0,-2370.0,22403.0,-0.47,-2.84,27.61,23.98,82.0,-0.11,0.56,2.82,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.88,44.0,X-SC,4.44,81977.0,-63183.0,139107.0,0.37,-43.53,169.69,52.31,136.0,-0.45,0.57,9.73,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.86,40.0,X-MC,2.51,88155.0,-2523.0,17428.0,1.21,-2.78,19.77,16.44,101.0,-0.14,0.61,16.83,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88852.0,-39818.0,85120.0,0.00,-30.95,95.80,35.20,219.0,-0.47,0.62,1.98,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-29.01,48.0,X-LC,10.05,283034.0,-62922.0,129630.0,-0.19,-18.19,45.80,19.28,1.0,-0.49,1.96,4.63,X40,ATH,IT
39,INFY,2275.00,-19.24,50.0,X-LC,5.24,314175.0,1287.0,170409.0,-0.71,0.41,54.24,54.87,3.0,0.01,2.18,8.73,X40,BTT,IT
41,ITC,452.00,-37.50,51.0,X-LC,1.16,201566.0,1428.0,19471.0,-0.42,0.71,9.66,10.44,4.0,0.07,1.40,6.32,X40,NTT,FMCG
83,VBL,671.64,-16.28,54.0,X-LC,7.18,301327.0,-14515.0,127190.0,-0.52,-4.60,42.21,35.67,5.0,-0.11,2.09,8.62,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,52.0,H-LC,5.22,251664.0,-9955.0,129179.0,-0.25,-3.81,51.33,45.57,7.0,-0.08,1.74,7.09,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,43.0,L-SC,28.09,78835.0,-34714.0,74767.0,-0.48,-30.57,94.84,35.27,268.0,-0.46,0.55,92.95,XR,NTT,HOTELS
7,ASIANTILES,137.00,7383.33,68.0,L-SC,9.76,82116.0,-11694.0,88308.0,-0.08,-12.47,107.54,81.67,269.0,-0.13,0.57,59.02,XR,NTT,CERAMICS
49,MASFIN,398.61,-16.44,60.0,H-SC,8.82,95190.0,-2790.0,24397.0,-0.91,-2.85,25.63,22.05,152.0,-0.11,0.66,38.37,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.65,126610.0,11812.0,82233.0,-0.15,10.29,64.95,81.92,223.0,0.14,0.88,49.89,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.07,130634.0,7298.0,124324.0,-0.08,5.92,95.17,106.72,119.0,0.06,0.91,30.68,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,81.27,74.0,H-SC,1.77,157356.0,-25419.0,136491.0,-0.80,-13.91,86.74,60.77,125.0,-0.19,1.09,40.94,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-51.56,77.0,H-SC,6.78,277698.0,-8807.0,266146.0,2.56,-3.07,95.84,89.82,134.0,-0.03,1.93,46.92,XY24,NTT,FINANCE
46,KPIGREEN,731.05,13.63,74.0,H-SC,6.55,135261.0,9964.0,48964.0,-0.38,7.95,36.20,47.03,141.0,0.20,0.94,69.36,MH,ATH,POWER
62,SAMMAANCAP,326.00,-200.00,78.0,M-SC,32.41,172485.0,23265.0,120912.0,0.91,15.59,70.10,96.62,208.0,0.19,1.20,86.67,XY25,NTT,FINANCE
85,WHIRLPOOL,2270.00,-38.90,59.0,M-SC,4.65,126610.0,11812.0,82233.0,-0.15,10.29,64.95,81.92,223.0,0.14,0.88,49.89,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,43.98
2,50,75.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.21
LC    32.88
MC    22.90
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.04
X40      16.06
X40N     12.45
XY25     11.90
XR        9.95
AR        7.81
OX40N     6.54
X200      1.77
X5K       1.44
SR        1.09
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.46
X-LC    22.22
X-MC    18.57
M-SC    12.75
X-SC     5.55
M-LC     5.10
H-LC     4.59
H-MC     2.35
M-MC     1.67
L-SC     1.45
L-LC     0.97
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.10,-3.96,37.57
IT,12.39,-17.53,80.50
FINANCE,12.28,-6.78,55.17
MISC,7.80,-15.53,73.81
ELECTRICAL,5.77,-8.53,47.01
PAINTS,5.44,-17.69,35.24
BANKS,4.05,-15.48,82.66
INSURANCE,3.81,-0.46,37.92
AUTO,2.76,-44.25,104.46


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3242352.0,22
XR,1219994.0,13
AR,1159947.0,9
X40,839742.0,12
X40N,673120.0,9
OX40N,540313.0,9
XY25,458639.0,8
SR,264820.0,2
X5K,110453.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3303799.0,24
M-SC,1206720.0,15
X-MC,1194613.0,14
X-LC,1075649.0,13
X-SC,549453.0,6
H-LC,275222.0,3
M-LC,272624.0,4
L-SC,252142.0,3
H-MC,249015.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1160057.0      6
           AR         830317.0      5
           XR         758727.0      7
M-SC       XY24       694231.0      6
X-MC       XY24       575777.0      4
X-LC       X40        466333.0      5
           XY24       304889.0      2
X-SC       X40N       299822.0      4
X-MC       X40        288289.0      6
H-SC       SR         264820.0      2
           OX40N      240914.0      3
X-LC       X40N       220951.0      3
H-LC       AR         207007.0      2
X-MC       XY25       178200.0      2
H-MC       XY24       178181.0      1
X-SC       XY24       164511.0      1
L-SC       XR         163075.0      2
X-MC       X40N       152347.0      2
M-MC       XY24       151217.0      1
M-SC       OX40N      139498.0      4
           XR         129456.0      2
           AR         122623.0      2
           XY25       120912.0      1
M-LC       XR         111564.0      1
           X5K        110453.0      1
L-SC       OX40N       89067.0      1
X-SC       X40         85120.0      1
X-LC       XY25        83476.0      3
H-MC       OX40N       70834.0      1
H-LC       X200        68215.0      1
L-MC       XR          57172.0      1
H-SC       MH          48964.0      1
L-LC       XY25        38933.0      1
M-LC       XY25        37118.0      1
           XY24        13489.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 58.0 seconds
